In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo

from sklearn.preprocessing import MinMaxScaler


In [6]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Definir la métrica personalizada
def multinacional_metric(y_true, y_pred):
    return abs(sum(y_true - y_pred)) / sum(y_true)
# Función para escalar y devolver una serie
def minmax_scale_group(group):
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Almacenar el escalador para este grupo
    return pd.Series(scaled_values, index=group.index)

# Función para desescalar y devolver una serie
def inverse_minmax_scale_group(group):
    scaler = scalers[group.name]
    inversed_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    return pd.Series(inversed_values, index=group.index)

# Leer datos
df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-10-01']


In [7]:

### Agrupar y escalar

scalers = {}

df_final['diff_tn_tn2'] = df_final.groupby('product_id')['diff_tn_tn2'].transform(minmax_scale_group) #escalado


In [8]:

# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric':'rmse',
        #'n_jobs': -1,
        #'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}
model, average_metric = labo.train_lightgbm_model(df_final,params,metric='rmse',col='diff_tn_tn2')
print("Overall custom metric: ", average_metric)




[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.242699 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 80119
[LightGBM] [Info] Number of data points in the train set: 237898, number of used features: 790
[LightGBM] [Info] Start training from score 0.474182
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[286]	validation's rmse: 0.0294864
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the 

In [9]:
def predict_next_month(model, last_data_points,col='diff_tn_tn2'):
    predictions = []
    last_month = last_data_points.index.max() + 1
    last_data_points.index = [last_month] * len(last_data_points)  # Set index to the next month
    
    predictions = model.predict(last_data_points, num_iteration=model.best_iteration)
    
    prediction_df = last_data_points[['product_id']].copy()
    prediction_df[col] = predictions
    prediction_df.index = [last_month] * len(last_data_points)

    return prediction_df

In [10]:
# Predict values for the entire dataset using the trained models
# Prepare last data points for prediction
last_data_points = df_final[df_final.index == df_final.index.max()].copy()
last_data_points.drop(columns=['diff_tn_tn2'], inplace=True)


In [22]:
df_final.index.max()

Period('2019-10', 'M')

In [26]:
# Predict the next month's value using the trained model
predictions = predict_next_month(model, last_data_points)

preds = predictions.groupby('product_id')['diff_tn_tn2'].transform(inverse_minmax_scale_group)
tn = df_final[df_final.index == df_final.index.max()].groupby('product_id')['tn'].transform(inverse_minmax_scale_group)
predictions['tn'] = preds + tn.values
predictions.drop(columns=['diff_tn_tn2'], inplace=True)
predictions = predictions.reset_index()
predictions =  predictions.groupby('product_id')['tn'].sum()
predictions.columns = ['product_id', 'tn']
predictions.to_csv(DATOS_DIR+'/pred/predicciones-timewarping-rmse.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


AttributeError: 'SeriesGroupBy' object has no attribute 'values'

In [25]:

print("Error: " ,abs(sum(df_true.groupby('product_id')['tn'].sum().values-predictions.values))/sum(df_true.groupby('product_id')['tn'].sum().values))

Error:  267.96525029210363
